In [3]:
import os

from dotenv import load_dotenv
from openai import OpenAI

In [4]:
# Load environment variables from .env file
load_dotenv()

True

# OpenAI

In [5]:
# OpenAI client
client = OpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [6]:
query = "Tell me a story about a wise rabbit and a clever fox."

stream = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": query}],
    stream=True,
)

for chunk in stream:
    print(chunk)

ChatCompletionChunk(id='', choices=[], created=0, model='', object='', service_tier=None, system_fingerprint=None, usage=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
ChatCompletionChunk(id='chatcmpl-Cq55d5XsW7cVHeFBmZR72GrxCGgDi', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1766528657, model='gpt-4.1-mini-2025-04-14', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_3dcd5944f5', usage=None, obfuscation='X8b18s18MNPXVm')
ChatCompletionChunk(id='chatcmpl-Cq55d5XsW7cVHeFBmZR72GrxCGgDi', choices=[Choice(delta=ChoiceDelta(conte

# LangChain

In [7]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

In [8]:
# initialize model using custom endpoint
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

agent = create_agent(llm)

In [11]:
query = "Tell me a story about a wise rabbit and a clever fox."

stream = agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="messages",
)

for chunk in stream:
    print(chunk)

(AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019b4d5b-fcd8-73a2-bdd5-0f59b3085469'), {'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:3cacebcf-1964-9756-f165-bce3d1d413a7', 'checkpoint_ns': 'model:3cacebcf-1964-9756-f165-bce3d1d413a7', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1-mini', 'ls_model_type': 'chat', 'ls_temperature': None})
(AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019b4d5b-fcd8-73a2-bdd5-0f59b3085469'), {'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:3cacebcf-1964-9756-f165-bce3d1d413a7', 'checkpoint_ns': 'model:3cacebcf-1964-9756-f165-bce3d1d413a7', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1-mini', 'ls_model_

In [30]:
def langchain_stream_to_streamlit(stream):
    for chunk, _ in stream:
        # Skip ToolMessages
        if chunk.type == "tool":
            continue
        # Only yield non-empty content
        elif hasattr(chunk, "content") and chunk.content:
            yield chunk.content

In [31]:
stream = agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="messages",
)

for chunk in langchain_stream_to_streamlit(stream):
    print(chunk)

876
5
 multiplied
 by
 
432
1
 is
 
37
,
873
,
565
.


## Tool calling

In [32]:
import sympy

In [33]:
# initialize model using custom endpoint
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
)


def calculate(expression: str) -> str:
    """Calculate a mathematical expression.
    Args:
        expression (str): Python arithmetic expression to calculate.
    Returns:
        str: The result of the calculation.
    """
    try:
        result = sympy.sympify(expression).evalf()
        return str(result)
    except Exception as e:
        return f"Error calculating expression: {e}"


# create the agent by combining the model and the function
agent = create_agent(
    model=llm,
    tools=[calculate],
    system_prompt="You are a helpful maths assistant. Use your calculator to help with maths problems.",
)

In [36]:
query = "What is 8765 multiplied by 4321?"

stream = agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="messages",
)

for item in stream:
    print(item)

(AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='lc_run--019b4d65-aed0-7760-9bc7-7319a40cc6e7'), {'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:251f97f2-894f-54ac-69c5-f4f02ac1a22a', 'checkpoint_ns': 'model:251f97f2-894f-54ac-69c5-f4f02ac1a22a', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1-mini', 'ls_model_type': 'chat', 'ls_temperature': None})
(AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019b4d65-aed0-7760-9bc7-7319a40cc6e7', tool_calls=[{'name': 'calculate', 'args': {}, 'id': 'call_j61xSWgcu4lrR7x4IkXvXmqM', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'calculate', 'args': '', 'id': 'call_j61xSWgcu4lrR7x4IkXvXmqM', 'index': 0, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph